# Notebook to process output from thecontentmine

This notebook illustrates the basic process of processing each resource in each paper and allocates a score.

In [1]:
import pandas
import process_urls

In [2]:
with open('amw_token', 'r') as f:
    github_token = f.read()

In [3]:
dict_of_papers = {'10.1016/j.cageo.2012.05.031': {'github': ['https://github.com/andreww/MSAT',
                                                             'https://github.com/url_that_does_not_resolve.com'],
                                                  'zenodo': ['https://zenodo.org/record/1196821#.WrtnltPwYkg']
                                                 }
                 }

In [4]:
resources_list = process_urls.process_papers_dict(dict_of_papers, verbose=True, 
                                                  github_token=github_token)

processing paper with doi 10.1016/j.cageo.2012.05.031
URL https://github.com/andreww/MSAT has a licence
URL https://github.com/url_that_does_not_resolve.com did not resolve
Paper with doi 10.1016/j.cageo.2012.05.031 has score of 1.0


In [5]:
print(resources_list)

[{'score': 2, 'doi': '10.1016/j.cageo.2012.05.031', 'licence_exists': True, 'url': 'https://github.com/andreww/MSAT', 'resolves': True, 'timestamp': '2018-03-28T13:48:48.520918', 'resourcetype': 'github'}, {'score': 0, 'doi': '10.1016/j.cageo.2012.05.031', 'licence_exists': None, 'url': 'https://github.com/url_that_does_not_resolve.com', 'resolves': False, 'timestamp': '2018-03-28T13:48:50.013805', 'resourcetype': 'github'}, {'score': 1, 'doi': '10.1016/j.cageo.2012.05.031', 'licence_exists': None, 'url': 'https://zenodo.org/record/1196821#.WrtnltPwYkg', 'resolves': True, 'timestamp': '2018-03-28T13:48:50.895424', 'resourcetype': 'github'}]


## Analysis of the data

We can store the data in a flat SQL database, but here we stuff it into a Pandas dataframe to allow some analysis.

In [6]:
url_df = pandas.DataFrame.from_dict(resources_list)

In [7]:
url_df

,doi,licence_exists,resolves,resourcetype,score,timestamp,url
0,10.1016/j.cageo.2012.05.031,True,True,github,2,2018-03-28T13:48:48.520918,https://github.com/andreww/MSAT
1,10.1016/j.cageo.2012.05.031,None,False,github,0,2018-03-28T13:48:50.013805,https://github.com/url_that_does_not_resolve.com
2,10.1016/j.cageo.2012.05.031,None,True,github,1,2018-03-28T13:48:50.895424,https://zenodo.org/record/1196821#.WrtnltPwYkg
